* **Jairo Nicolás Gómez**
* **Juan Pablo Martinez**

# **20N**

## **1) Read files**

In [1]:
import os
root_folder = "./20news-18828"
# Read files and write them in a txt called output.txt
def read_files_in_folder(folder_path):
    lista = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if os.path.isfile(file_path):
                try:
                    with open(file_path, 'r', encoding='latin-1') as file:
                        file_contents = file.read().replace('\n', ' ').replace('\t', ' ')
                        lista.append(file_contents)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
    
    diccionario = {}
    for index,words in enumerate(lista):
        sentences = words.split(".")
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        new_list = []
        for ind in sentences:
            if len(ind) > 10:
                new_list.append(ind)
                diccionario[index] = new_list
    return diccionario

dictionary_documents_20news = read_files_in_folder(root_folder)


In [2]:
print(dictionary_documents_20news[12345])


['From: baden@sys6626', "ca (baden de bari) Subject: !!!] HELP!  NEED 3479P!!! [!!!                   Would anyone have a few extra 3479P's lying around that I  could buy off of them", "Problem is that around here I can only perchase  them in $30 quantities, and I don't need this, and can't take this  financially right now", 'If anyone can accomodate me with this PLEASE  reply to BOTH for the following mailing addresses', 'Thanks!!!   _______________________________________________                  |                                 |    _______   |    Baden de Bari                |   /       \\  |     baden@sys6626', 'ca   |  (| o   o |) |     baden@inqmind', 'ca   |   |   ^   |  | >> True life can only           |   \\  -=-  /  | >> be experianced by            |    \\_____/   | >> those who do not fear death', '| -----------------------------------------------']


## **2)Tokenize by sentence**

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import nltk
from collections import Counter

#Loop through the dictionary of files and do the preproccessing. Return list of sentences
def preprocess(documents):

    for index,keys in enumerate(documents):
        sentences = documents[keys]
        #Eliminate punctuation
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        word_tok_nltk = [tokenizer.tokenize(doc) for doc in sentences]
        #Stemming
        p_stemmer = nltk.stem.porter.PorterStemmer()
        nltk_stemedList = [[p_stemmer.stem(word) for word in doc] for doc in word_tok_nltk]
        #Replace number with token NUM
        big_list = []
        for lists in nltk_stemedList:
            lists.insert(0, "<s>")
            lists.append("</s>")
            numbers = []
            for strings in lists:
                try:
                    turn_number = int(strings)
                    numbers.append("NUM")
                except:
                    numbers.append(strings)
            big_list.append(numbers)           
        documents[keys] = big_list

    all_sentences = []
    for key in documents:
        list_of_lists = documents[key]
        for sentences in list_of_lists:
            all_sentences.append(sentences)
                             
    flattened_list = [item for sublist in all_sentences for item in sublist] #List of words separated

    counted = Counter(flattened_list)

    frequency_threshold = 2

    word_frequencies_dict = dict(counted) #Dictionary with the frequencies fo every word in the corpus

    final_sentences_result = []
    dynamic_list = []
    string = ""
    for i in flattened_list:
        if word_frequencies_dict[i] >= 2:
            if i == "</s>":
                dynamic_list.append(i)
                string = " ".join(dynamic_list)
                final_sentences_result.append(string)
                dynamic_list = []
                string = ""
            
            else:
                dynamic_list.append(i)
        else:
            dynamic_list.append('<UNK>')
            
            
    return final_sentences_result

procecessed_sentences = preprocess(dictionary_documents_20news)

In [5]:
print(len(procecessed_sentences))

357587


## **3) Training 80% and Test 20%**

In [6]:
import random

def divide_training_test(list_sentences):

    training_number = int(round((len(list_sentences)*80)/100,0))
    filename_training = "20N_training.txt"
    list_training = []


    test_number = int(round((len(list_sentences)*20/100),0))
    filename_test = "20N_testing.txt"
    list_test = []

    count = len(list_sentences)-1
    while training_number != 0:
        index_sentence = random.randint(0,count)
        sentence_training = list_sentences.pop(index_sentence)
        list_training.append(sentence_training)
        count -= 1
        training_number -= 1

    #Write in the training file
    with open(filename_training, 'w') as file:
        for sentence in list_training:
            file.write(sentence + '\n')
        
    while test_number != 0:
        index_sentence = random.randint(0,count)
        sentence_training = list_sentences.pop(index_sentence)
        list_test.append(sentence_training)
        count -= 1
        test_number -= 1
        
    #Write in the training file
    with open(filename_test, 'w') as file:
        for sentence in list_test:
            file.write(sentence + '\n')
    

divide_training_test(procecessed_sentences)
    

## **N-Gram Models Laplace Smoothing**

### **Unigrams**

In [7]:
#Create the N-Gram model for Unigrams
def unigram():
    corpus = []
    with open('20N_training.txt', 'r') as file:
        for line in file:
            line = line.strip()
            list = line.split()
            corpus.append(list)
    flattened_list = [item for sublist in corpus for item in sublist] #List of words separated
    
    word_counts = Counter(flattened_list) #Dictionary with the ocurrences of each word from the new corpus
    N = len(flattened_list) #Number of words from the corpus
    V = len(word_counts) # Vocabulary
    smoothed_probabilities = {}
    smoothed_prob = 0
    # Laplace smoothing
    for word, count in word_counts.items():
        #if word != "<UNK>":
        smoothed_prob = (count + 1) / (N + V)
        smoothed_probabilities[word] = smoothed_prob

    filename_unigrams = "20N_unigrams.txt"
    with open(filename_unigrams, 'w') as file:
        for word, prob in smoothed_probabilities.items():
            file.write(f"'{word}': {prob}" + '\n')


unigram()
            

### **Bigrams**

In [8]:
#Create the N-Gram model for Bigrams
def bigrams():
    corpus = []
    with open('20N_training.txt', 'r') as file:
        for line in file:
            line = line.strip()
            list = line.split()
            corpus.append(list)

    individual_words = [item for sublist in corpus for item in sublist] #List of words separated
    unigram_counts = Counter(individual_words)

    bigrams = []
    for sentence in corpus:
        for word in range(1,len(sentence)):
            #if sentence[word] != "<UNK>" and sentence[word-1] != "<UNK>":
            bigrams.append((sentence[word-1], sentence[word]))
            
    bigram_counts = Counter(bigrams)
    
    V = len(bigram_counts) # Vocabulary

    smoothing_probabilities = {}

    for bigram, frequency in bigram_counts.items():

        firsterm, secondterm = bigram

        unigram_counter = unigram_counts[firsterm]

        smoothing_freq = (frequency + 1)/(unigram_counter + V)
        smoothing_probabilities[bigram] = smoothing_freq

        
    filename_bigrams = "20N_bigrams.txt"
    with open(filename_bigrams, 'w') as file:
        for word, prob in smoothing_probabilities.items():
            file.write(f"'{word}': {prob}" + '\n')
    

bigrams()
 

### **Trigrams**

In [9]:
#Create the N-Gram model for Trigrams
def trigrams():
    corpus = []
    with open('20N_training.txt', 'r') as file:
        for line in file:
            line = line.strip()
            list = line.split()
            corpus.append(list)
            
    bigrams = []
    for sentence in corpus:
        for word in range(1,len(sentence)):
            #if sentence[word] != "<UNK>" and sentence[word-1] != "<UNK>":
            bigrams.append((sentence[word-1], sentence[word]))
            
    bigram_counts = Counter(bigrams)

    trigrams = []
    for sentence in corpus:
        for word in range(2,len(sentence)):
            #if sentence[word] != "<UNK>" and sentence[word-1] != "<UNK>" and sentence[word-2] != "<UNK>":
            trigrams.append((sentence[word-2],sentence[word-1], sentence[word]))

    trigrams_count = Counter(trigrams)

    V = len(trigrams_count)

    trigrams_probabilities = {}

    for trigrams, frequency in trigrams_count.items():
        firstword, secondword, thirdword = trigrams
        bigram_count = bigram_counts[(firstword,secondword)]
        smoothing = (frequency + 1) / (bigram_count+V)
        trigrams_probabilities[trigrams] = smoothing

    filename_trigrams = "20N_trigrams.txt"
    with open(filename_trigrams, 'w') as file:
        for word, prob in trigrams_probabilities.items():
            file.write(f"'{word}': {prob}" + '\n')

trigrams()    

# **Blogs**

### **Read files and preprocess**

In [10]:
!pip install lxml


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from lxml import etree
from os import scandir
from re import sub
import math

In [12]:
# Crear una carpeta llamada "blogs" que incluya todos los archivos .xml
directory = 'blogs'

with open('BAC_final_file.txt', 'w', errors='ignore') as bf:
    for entry in scandir(directory):
        if entry.is_file():
            with open(entry.path, 'r', encoding='unicode_escape', errors='ignore') as f:
                doc_content = f.read()
                parser = etree.XMLParser(recover=True)
                root = etree.fromstring(doc_content, parser=parser)
                post_elements = root.findall(".//post")
                for post_element in post_elements:
                    post_content = post_element.text
                    lower_string = post_content.lower() # Colocar todo el texto en minusculas
                    no_number_string = sub(r'\d+','NUM',lower_string) # Quitar numeros
                    no_punc_string = sub(r'[^\w\s]','', no_number_string) # Quitar signos de puntuacion
                    no_wspace_string = no_punc_string.strip() # Quitar espacios al principio y final
                    final_string = no_wspace_string.replace('\n','')
                    bf.write(f'<s> {final_string} </s>\n')

In [13]:
# Leer el archivo consolidado para dividir los subconjuntos de entrenamiento y testing
file_content = []
with open('BAC_final_file.txt', 'r') as tf:
    file_content = tf.readlines()

# Cacular cuanto es el 80 % de los datos para generar el archivo de entrenamiento
sentence_num = len(file_content)
p80 = math.floor(sentence_num * 0.8)

# Generar el archivo de training
with open('BAC_training.txt', 'w') as btr:
    for i in range(p80):
        btr.write(file_content[i])

# Generar el archivo de testing
with open('BAC_testing.txt', 'w') as bts:
    for i in range(p80+1, sentence_num):
        bts.write(file_content[i])

### **Unigrams**

In [14]:
#Create the N-Gram model for Unigrams
def unigram_bac():
    corpus = []
    with open('BAC_training.txt', 'r') as file:
        for line in file:
            line = line.strip()
            list = line.split()
            corpus.append(list)
    flattened_list = [item for sublist in corpus for item in sublist] #List of words separated
    
    word_counts = Counter(flattened_list) #Dictionary with the ocurrences of each word from the new corpus
    N = len(flattened_list) #Number of words from the corpus
    V = len(word_counts) # Vocabulary
    smoothed_probabilities = {}
    smoothed_prob = 0
    # Laplace smoothing
    for word, count in word_counts.items():
        #if word != "<UNK>":
        smoothed_prob = (count + 1) / (N + V)
        smoothed_probabilities[word] = smoothed_prob

    filename_unigrams = "BAC_unigrams.txt"
    with open(filename_unigrams, 'w') as file:
        for word, prob in smoothed_probabilities.items():
            file.write(f"'{word}': {prob}" + '\n')


unigram_bac()
            

### **Bigrams**

In [ ]:
#Create the N-Gram model for Bigrams
def bigrams_bac():
    corpus = []
    with open('BAC_training.txt', 'r') as file:
        for line in file:
            line = line.strip()
            list = line.split()
            corpus.append(list)

    individual_words = [item for sublist in corpus for item in sublist] #List of words separated
    unigram_counts = Counter(individual_words)

    bigrams = []
    for sentence in corpus:
        for word in range(1,len(sentence)):
            #if sentence[word] != "<UNK>" and sentence[word-1] != "<UNK>":
            bigrams.append((sentence[word-1], sentence[word]))
            
    bigram_counts = Counter(bigrams)
    
    V = len(bigram_counts) # Vocabulary

    smoothing_probabilities = {}

    for bigram, frequency in bigram_counts.items():

        firsterm, secondterm = bigram

        unigram_counter = unigram_counts[firsterm]

        smoothing_freq = (frequency + 1)/(unigram_counter + V)
        smoothing_probabilities[bigram] = smoothing_freq

        
    filename_bigrams = "BAC_bigrams.txt"
    with open(filename_bigrams, 'w') as file:
        for word, prob in smoothing_probabilities.items():
            file.write(f"'{word}': {prob}" + '\n')
    

bigrams_bac()
 

### **Trigrams**

In [ ]:
#Create the N-Gram model for Trigrams
def trigrams_bac():
    corpus = []
    with open('BAC_training.txt', 'r') as file:
        for line in file:
            line = line.strip()
            list = line.split()
            corpus.append(list)
            
    bigrams = []
    for sentence in corpus:
        for word in range(1,len(sentence)):
            #if sentence[word] != "<UNK>" and sentence[word-1] != "<UNK>":
            bigrams.append((sentence[word-1], sentence[word]))
            
    bigram_counts = Counter(bigrams)

    trigrams = []
    for sentence in corpus:
        for word in range(2,len(sentence)):
            #if sentence[word] != "<UNK>" and sentence[word-1] != "<UNK>" and sentence[word-2] != "<UNK>":
            trigrams.append((sentence[word-2],sentence[word-1], sentence[word]))

    trigrams_count = Counter(trigrams)

    V = len(trigrams_count)

    trigrams_probabilities = {}

    for trigrams, frequency in trigrams_count.items():
        firstword, secondword, thirdword = trigrams
        bigram_count = bigram_counts[(firstword,secondword)]
        smoothing = (frequency + 1) / (bigram_count+V)
        trigrams_probabilities[trigrams] = smoothing

    filename_trigrams = "BAC_trigrams.txt"
    with open(filename_trigrams, 'w') as file:
        for word, prob in trigrams_probabilities.items():
            file.write(f"'{word}': {prob}" + '\n')

trigrams_bac()    